# Advanced RVC Inference - No UI Version

This notebook provides a streamlined, command-line style interface for RVC inference with enhanced features:
- Model download and management
- YouTube audio download with cookie support
- Better parameter organization
- Batch processing capabilities
- Audio preview before and after conversion
- Preset configurations

In [ ]:
# Installation and Setup
import os
import sys
import subprocess
from pathlib import Path
import json
import time
import glob
import shutil
import warnings
import regex as re
warnings.filterwarnings('ignore')

print("Setting up Advanced RVC environment...")

# Check if we are in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab")
except:
    IN_COLAB = False
    print("Running in local environment")

# Install dependencies if requirements.txt exists
if os.path.exists("requirements.txt"):
    print("Installing dependencies...")
    subprocess.run(["pip", "install", "-r", "requirements.txt"], check=True)
else:
    print("requirements.txt not found, skipping dependency installation")

# Install yt-dlp for YouTube downloads
try:
    import yt_dlp
    print("yt-dlp is already installed")
except ImportError:
    print("Installing yt-dlp...")
    subprocess.run(["pip", "install", "yt-dlp"], check=True)
    import yt_dlp

# Download prerequisites if the script exists
if os.path.exists("programs/applio_code/rvc/lib/tools/prerequisites_download.py"):
    print("Downloading prerequisites...")
    subprocess.run(["python", "programs/applio_code/rvc/lib/tools/prerequisites_download.py"], check=True)
else:
    print("Prerequisites download script not found, skipping")

# Create necessary directories
os.makedirs("models", exist_ok=True)
os.makedirs("logs", exist_ok=True)
os.makedirs("assets", exist_ok=True)
os.makedirs("audio_files/input", exist_ok=True)
os.makedirs("audio_files/output", exist_ok=True)
os.makedirs("presets", exist_ok=True)

# Create cookies file if it does not exist
if not os.path.exists("./assets/ytdlstuff.txt"):
    with open("./assets/ytdlstuff.txt", "w") as f:
        f.write("# YouTube cookies file\n# Add your cookies here if needed\n")
    print("Created cookies file at ./assets/ytdlstuff.txt")

# Import required modules
try:
    from core import download_model, full_inference_program
    from programs.applio_code.rvc.lib.utils import format_title
    from assets.i18n.i18n import I18nAuto
    import torch
    import numpy as np
    import soundfile as sf
    from IPython.display import Audio, display, HTML
    import matplotlib.pyplot as plt
    from tqdm.notebook import tqdm
    
    # Initialize i18n
    i18n = I18nAuto()
    
    print("All modules imported successfully")
except ImportError as e:
    print(f"Error importing modules: {e}")
    print("Please ensure all dependencies are installed")

# Check GPU availability
if torch.cuda.is_available():
    print(f"GPU available: {torch.cuda.get_device_name(0)}")
    devices = "0"
else:
    print("No GPU available, using CPU")
    devices = "cpu"

print("Setup completed successfully!")

In [ ]:
# Model Management Utilities

def save_drop_model(file_path):
    """Save a dropped model file to the appropriate directory"""
    if file_path is None or ("pth" not in file_path and "index" not in file_path):
        print("Error: The file is not a valid model file. Please try again.")
        return False
    else:
        file_name = format_title(os.path.basename(file_path))
        if ".pth" in file_path:
            model_name = format_title(file_name.split(".pth")[0])
        else:
            if (
                "v2" not in file_path
                and "added_" not in file_path
                and "_nprobe_1_" not in file_path
            ):
                model_name = format_title(file_name.split(".index")[0])
            else:
                if "v2" not in file_path:
                    if "_nprobe_1_" in file_name and "_v1" in file_name:
                        model_name = format_title(
                            file_name.split("_nprobe_1_")[1].split("_v1")[0]
                        )
                    elif "added_" in file_name and "_v1" in file_name:
                        model_name = format_title(
                            file_name.split("added_")[1].split("_v1")[0]
                        )
                else:
                    if "_nprobe_1_" in file_name and "_v2" in file_name:
                        model_name = format_title(
                            file_name.split("_nprobe_1_")[1].split("_v2")[0]
                        )
                    elif "added_" in file_name and "_v2" in file_name:
                        model_name = format_title(
                            file_name.split("added_")[1].split("_v2")[0]
                        )

        model_name = re.sub(r"\d+[se]", "", model_name)
        if "__" in model_name:
            model_name = model_name.replace("__", "")

        model_path = os.path.join(os.getcwd(), "logs", model_name)
        os.makedirs(model_path, exist_ok=True)
        if os.path.exists(os.path.join(model_path, file_name)):
            os.remove(os.path.join(model_path, file_name))
        shutil.copy(file_path, os.path.join(model_path, file_name))
        print(f"{file_name} saved in {model_path}")
        return True

def download_model_from_url(url):
    """Download a model from a URL"""
    try:
        print(f"Downloading model from: {url}")
        result = download_model(url)
        print(f"Download result: {result}")
        return result
    except Exception as e:
        print(f"Error downloading model: {e}")
        return None

def list_available_models():
    """List all available RVC models"""
    models_path = "models"
    logs_path = "logs"
    
    model_files = []
    
    # Check models directory
    if os.path.exists(models_path):
        pth_files = glob.glob(os.path.join(models_path, "*.pth"))
        model_files.extend(pth_files)
    
    # Check logs directory (for downloaded models)
    if os.path.exists(logs_path):
        for root, dirs, files in os.walk(logs_path):
            for file in files:
                if file.endswith(".pth"):
                    model_files.append(os.path.join(root, file))
    
    if not model_files:
        print("No .pth model files found")
        return []
    
    print("Available models:")
    for i, path in enumerate(model_files, 1):
        name = os.path.basename(path)
        print(f"{i}. {name}")
    
    return model_files

def find_index_file(model_path):
    """Find the corresponding index file for a model"""
    model_dir = os.path.dirname(model_path)
    model_name = os.path.splitext(os.path.basename(model_path))[0]
    
    possible_extensions = [".index", ".npy", ".bin"]
    
    # First check in the same directory as the model
    for ext in possible_extensions:
        index_path = os.path.join(model_dir, f"{model_name}{ext}")
        if os.path.exists(index_path):
            return index_path
    
    # Check in models directory
    for ext in possible_extensions:
        index_path = os.path.join("models", f"{model_name}{ext}")
        if os.path.exists(index_path):
            return index_path
    
    # Try to find any index file with similar name
    index_files = glob.glob(os.path.join(model_dir, f"{model_name}*"))
    index_files = [f for f in index_files if any(f.endswith(ext) for ext in possible_extensions)]
    
    if index_files:
        return index_files[0]
    
    # Check in models directory for similar names
    index_files = glob.glob(os.path.join("models", f"{model_name}*"))
    index_files = [f for f in index_files if any(f.endswith(ext) for ext in possible_extensions)]
    
    if index_files:
        return index_files[0]
    
    return None

def upload_model():
    """Upload a model file (for Colab)"""
    if not IN_COLAB:
        print("This function is only available in Google Colab")
        return
    
    from google.colab import files
    uploaded = files.upload()
    
    for filename in uploaded.keys():
        if filename.endswith(".pth"):
            save_drop_model(filename)
        elif any(filename.endswith(ext) for ext in [".index", ".npy", ".bin"]):
            save_drop_model(filename)
        else:
            print(f"Skipping unsupported file: {filename}")

# List available models
available_models = list_available_models()

In [ ]:
# YouTube Audio Downloader

def download_youtube_audio(url, output_format="wav", output_dir="audio_files/input"):
    """Download audio from YouTube URL"""
    os.makedirs(output_dir, exist_ok=True)
    
    # Cookie file path
    cookies_path = "./assets/ytdlstuff.txt"
    
    # Configure yt-dlp options
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': output_format,
            'preferredquality': '192',
        }],
        'outtmpl': os.path.join(output_dir, '%(title)s.%(ext)s'),
        'quiet': False,
        'no_warnings': False,
    }
    
    # Add cookies if file exists and is not empty
    if os.path.exists(cookies_path) and os.path.getsize(cookies_path) > 0:
        ydl_opts['cookiefile'] = cookies_path
        print(f"Using cookies from {cookies_path}")
    else:
        print("No cookies found or cookie file is empty")
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            title = info.get('title', 'Unknown')
            
            # Find the downloaded file
            downloaded_file = None
            for file in os.listdir(output_dir):
                if title.replace('/', '_').replace('\\', '_') in file and file.endswith(f".{output_format}"):
                    downloaded_file = os.path.join(output_dir, file)
                    break
            
            if downloaded_file:
                print(f"Successfully downloaded: {downloaded_file}")
                return downloaded_file
            else:
                print("Error: Could not find downloaded file")
                return None
    except Exception as e:
        print(f"Error downloading YouTube audio: {e}")
        return None

def list_audio_files(directory):
    """List all audio files in a directory"""
    if not os.path.exists(directory):
        print(f"Directory not found: {directory}")
        return []
    
    audio_extensions = [".wav", ".mp3", ".flac", ".m4a", ".ogg"]
    audio_files = []
    
    for ext in audio_extensions:
        audio_files.extend(glob.glob(os.path.join(directory, f"*{ext}")))
    
    if not audio_files:
        print(f"No audio files found in {directory}")
        return []
    
    print(f"Found {len(audio_files)} audio files in {directory}:")
    for i, file in enumerate(audio_files, 1):
        print(f"{i}. {os.path.basename(file)}")
    
    return audio_files

# List available input audio files
input_audio_files = list_audio_files("audio_files/input")

In [ ]:
# Preset Configurations

def save_preset(name, config):
    """Save a configuration preset"""
    preset_path = os.path.join("presets", f"{name}.json")
    with open(preset_path, 'w') as f:
        json.dump(config, f, indent=2)
    print(f"Preset '{name}' saved to {preset_path}")

def load_preset(name):
    """Load a configuration preset"""
    preset_path = os.path.join("presets", f"{name}.json")
    if not os.path.exists(preset_path):
        print(f"Preset '{name}' not found")
        return None
    
    with open(preset_path, 'r') as f:
        config = json.load(f)
    print(f"Preset '{name}' loaded")
    return config

def list_presets():
    """List all available presets"""
    preset_files = glob.glob(os.path.join("presets", "*.json"))
    preset_names = [os.path.splitext(os.path.basename(f))[0] for f in preset_files]
    
    if not preset_names:
        print("No presets found")
        return []
    
    print("Available presets:")
    for i, name in enumerate(preset_names, 1):
        print(f"{i}. {name}")
    
    return preset_names

# Create default presets if they do not exist
if not os.path.exists("presets/high_quality.json"):
    high_quality_preset = {
        "pitch": 0,
        "index_rate": 0.8,
        "rms_mix_rate": 0.25,
        "protect": 0.33,
        "hop_length": 128,
        "filter_radius": 3,
        "f0_method": "rmvpe",
        "split_audio": True,
        "autotune": False,
        "denoise": True,
        "export_format": "wav"
    }
    save_preset("high_quality", high_quality_preset)

if not os.path.exists("presets/fast_conversion.json"):
    fast_preset = {
        "pitch": 0,
        "index_rate": 0.7,
        "rms_mix_rate": 0.5,
        "protect": 0.33,
        "hop_length": 256,
        "filter_radius": 3,
        "f0_method": "harvest",
        "split_audio": False,
        "autotune": False,
        "denoise": False,
        "export_format": "wav"
    }
    save_preset("fast_conversion", fast_preset)

# List available presets
available_presets = list_presets()

In [ ]:
# Configuration Parameters

class RVCConfig:
    """Configuration class for RVC inference"""
    def __init__(self):
        # Model settings
        self.model_path = ""  # Path to your RVC model
        self.index_path = ""  # Path to your index file
        self.model_name = "contentvec"  # or other model types
        
        # Audio settings
        self.input_audio_path = ""  # Your input audio file
        self.output_path = "audio_files/output/output.wav"  # Output file path
        
        # Conversion parameters
        self.pitch = 0  # Pitch adjustment in semitones
        self.index_rate = 0.75  # How much to use the index file (0.0-1.0)
        self.rms_mix_rate = 0.25  # RMS mix rate (0.0-1.0)
        self.protect = 0.33  # Protection for voiceless consonants (0.0-1.0)
        self.hop_length = 128  # Hop length for audio processing
        self.filter_radius = 3  # Filter radius
        self.split_audio = True  # Whether to split audio for processing large files
        self.autotune = False  # Apply autotune
        self.f0_method = "rmvpe"  # Method for pitch extraction ('harvest', 'crepe', 'rmvpe', etc.)
        
        # Processing options
        self.vocal_model = "MDX23C"  # Vocal separation model
        self.karaoke_model = "UVR-BVE"  # Karaoke model
        self.dereverb_model = "UVR-Deecho-Dereverb"  # Dereverb model
        self.deecho = False  # Apply deecho
        self.denoise = False  # Apply denoising
        self.reverb = False  # Apply reverb
        
        # Output format
        self.export_format = "wav"  # Output format ('wav', 'mp3', 'flac')
        
        # Batch processing
        self.batch_mode = False  # Enable batch processing
        self.batch_input_dir = "audio_files/input"  # Directory with input files for batch processing
        self.batch_output_dir = "audio_files/output"  # Directory for batch output
        
        # Advanced options
        self.use_tta = False  # Use Test Time Augmentation
        self.batch_size = 1  # Batch size for processing
        self.delete_audios = False  # Delete intermediate audio files
        
        # Device settings
        self.devices = devices  # Set based on GPU availability
    
    def update_from_preset(self, preset_name):
        """Update configuration from a preset"""
        preset = load_preset(preset_name)
        if preset:
            for key, value in preset.items():
                if hasattr(self, key):
                    setattr(self, key, value)
            print(f"Configuration updated from preset '{preset_name}'")
            return True
        return False
    
    def set_model(self, model_path):
        """Set the model and find the corresponding index file"""
        if os.path.exists(model_path):
            self.model_path = model_path
            self.index_path = find_index_file(model_path)
            print(f"Model set to: {model_path}")
            if self.index_path:
                print(f"Index file found: {self.index_path}")
            else:
                print("No index file found")
            return True
        else:
            print(f"Model file not found: {model_path}")
            return False
    
    def set_input_audio(self, audio_path):
        """Set the input audio file"""
        if os.path.exists(audio_path):
            self.input_audio_path = audio_path
            print(f"Input audio set to: {audio_path}")
            return True
        else:
            print(f"Input audio file not found: {audio_path}")
            return False
    
    def print_config(self):
        """Print the current configuration"""
        print("Current Configuration:")
        print(f"  Model: {self.model_path}")
        print(f"  Index: {self.index_path}")
        print(f"  Input Audio: {self.input_audio_path}")
        print(f"  Output Path: {self.output_path}")
        print(f"  Pitch: {self.pitch}")
        print(f"  F0 Method: {self.f0_method}")
        print(f"  Index Rate: {self.index_rate}")
        print(f"  Batch Mode: {self.batch_mode}")
        print(f"  Device: {self.devices}")

# Initialize configuration
config = RVCConfig()
print("Configuration initialized")
config.print_config()

In [ ]:
# Audio Utilities

def upload_audio():
    """Upload audio files (for Colab)"""
    if not IN_COLAB:
        print("This function is only available in Google Colab")
        return
    
    from google.colab import files
    uploaded = files.upload()
    
    audio_extensions = [".wav", ".mp3", ".flac", ".m4a", ".ogg"]
    
    for filename in uploaded.keys():
        if any(filename.lower().endswith(ext) for ext in audio_extensions):
            shutil.move(filename, os.path.join("audio_files/input", filename))
            print(f"Audio file {filename} uploaded to audio_files/input directory")
        else:
            print(f"Skipping unsupported file: {filename}")

def preview_audio(audio_path, title="Audio Preview"):
    """Display an audio player with visualization"""
    if not os.path.exists(audio_path):
        print(f"Audio file not found: {audio_path}")
        return
    
    try:
        # Load audio file
        audio_data, sample_rate = sf.read(audio_path)
        
        # Create waveform plot
        plt.figure(figsize=(12, 3))
        time_axis = np.linspace(0, len(audio_data) / sample_rate, num=len(audio_data))
        plt.plot(time_axis, audio_data)
        plt.title(f"{title} - Waveform")
        plt.xlabel("Time (s)")
        plt.ylabel("Amplitude")
        plt.grid(True)
        plt.tight_layout()
        plt.show()
        
        # Display audio player
        display(Audio(audio_path, autoplay=False))
        
        # Print audio info
        duration = len(audio_data) / sample_rate
        print(f"Audio Info:")
        print(f"- Duration: {duration:.2f} seconds")
        print(f"- Sample Rate: {sample_rate} Hz")
        print(f"- Channels: {1 if len(audio_data.shape) == 1 else audio_data.shape[1]}")
    except Exception as e:
        print(f"Error previewing audio: {e}")
        # Fallback to just display the audio player
        try:
            display(Audio(audio_path, autoplay=False))
        except Exception as e2:
            print(f"Could not display audio player: {e2}")

def convert_audio_format(input_path, output_path, target_format="wav"):
    """Convert audio to a different format"""
    try:
        audio_data, sample_rate = sf.read(input_path)
        sf.write(output_path, audio_data, sample_rate, format=target_format)
        print(f"Audio converted to {target_format}: {output_path}")
        return True
    except Exception as e:
        print(f"Error converting audio: {e}")
        return False

In [ ]:
# Helper Functions

def validate_config(config):
    """Validate the configuration parameters"""
    errors = []
    warnings = []
    
    # Check model file
    if not config.model_path or not os.path.exists(config.model_path):
        errors.append(f"Model file not found: {config.model_path}")
    
    # Check index file
    if config.index_path and not os.path.exists(config.index_path):
        warnings.append(f"Index file not found: {config.index_path}")
    
    # Check input audio file
    if not config.batch_mode and (not config.input_audio_path or not os.path.exists(config.input_audio_path)):
        errors.append(f"Input audio file not found: {config.input_audio_path}")
    
    # Check batch input directory
    if config.batch_mode and not os.path.exists(config.batch_input_dir):
        errors.append(f"Batch input directory not found: {config.batch_input_dir}")
    
    # Validate parameter ranges
    if not 0 <= config.index_rate <= 1:
        errors.append(f"index_rate must be between 0 and 1, got {config.index_rate}")
    
    if not 0 <= config.rms_mix_rate <= 1:
        errors.append(f"rms_mix_rate must be between 0 and 1, got {config.rms_mix_rate}")
    
    if not 0 <= config.protect <= 1:
        errors.append(f"protect must be between 0 and 1, got {config.protect}")
    
    # Validate f0_method
    valid_f0_methods = ["harvest", "crepe", "rmvpe", "dio", "mangio-crepe", "mangio-dio"]
    if config.f0_method not in valid_f0_methods:
        warnings.append(f"f0_method '{config.f0_method}' may not be supported. Valid options: {valid_f0_methods}")
    
    # Validate export format
    valid_formats = ["wav", "mp3", "flac"]
    if config.export_format not in valid_formats:
        warnings.append(f"export_format '{config.export_format}' may not be supported. Valid options: {valid_formats}")
    
    # Print validation results
    if errors:
        print("❌ Configuration errors found:")
        for error in errors:
            print(f"  - {error}")
        return False
    
    if warnings:
        print("⚠️ Configuration warnings:")
        for warning in warnings:
            print(f"  - {warning}")
    
    print("✅ Configuration validation passed")
    return True

def run_conversion(config, input_file=None, output_file=None, progress_callback=None):
    """Run the RVC conversion with the specified parameters"""
    # Use provided files or defaults from config
    input_path = input_file if input_file else config.input_audio_path
    output_path = output_file if output_file else config.output_path
    
    # Check if files exist
    if not os.path.exists(input_path):
        print(f"Input file not found: {input_path}")
        return None
    
    if not os.path.exists(config.model_path):
        print(f"Model file not found: {config.model_path}")
        return None
    
    # Create output directory if it does not exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    print(f"Starting RVC conversion...")
    print(f"Input: {input_path}")
    print(f"Output: {output_path}")
    print(f"Model: {config.model_path}")
    print(f"Pitch: {config.pitch}")
    print(f"F0 Method: {config.f0_method}")
    
    start_time = time.time()
    
    try:
        if progress_callback:
            progress_callback(0.1, "Initializing conversion...")
        
        result = full_inference_program(
            model_path=config.model_path,
            index_path=config.index_path,
            input_audio_path=input_path,
            output_path=output_path,
            export_format_rvc=config.export_format,
            split_audio=config.split_audio,
            autotune=config.autotune,
            vocal_model=config.vocal_model,
            karaoke_model=config.karaoke_model,
            dereverb_model=config.dereverb_model,
            deecho=config.deecho,
            deecho_model="UVR-Deecho-Normal",
            denoise=config.denoise,
            denoise_model="UVR Denoise",
            reverb=config.reverb,
            vocals_volume=0.0,
            instrumentals_volume=0.0,
            backing_vocals_volume=0.0,
            export_format_final=config.export_format,
            devices=config.devices,
            pitch=config.pitch,
            filter_radius=config.filter_radius,
            index_rate=config.index_rate,
            rms_mix_rate=config.rms_mix_rate,
            protect=config.protect,
            pitch_extract=config.f0_method,
            hop_lenght=config.hop_length,
            reverb_room_size=0.15,
            reverb_damping=0.7,
            reverb_wet_gain=0.1,
            reverb_dry_gain=0.8,
            reverb_width=1.0,
            embedder_model=config.model_name,
            delete_audios=config.delete_audios,
            use_tta=config.use_tta,
            batch_size=config.batch_size,
            infer_backing_vocals=False,
            infer_backing_vocals_model="",
            infer_backing_vocals_index="",
            change_inst_pitch=0,
            pitch_back=0,
            filter_radius_back=3,
            index_rate_back=0.75,
            rms_mix_rate_back=0.25,
            protect_back=0.33,
            pitch_extract_back="harvest",
            hop_length_back=128,
            export_format_rvc_back=config.export_format,
            split_audio_back=False,
            autotune_back=False,
            embedder_model_back=config.model_name,
        )
        
        if progress_callback:
            progress_callback(1.0, "Conversion completed!")
        
        elapsed_time = time.time() - start_time
        print(f"Conversion completed in {elapsed_time:.2f} seconds")
        print(f"Output file: {result[1]}")
        return result[1]
    except Exception as e:
        print(f"Conversion failed: {e}")
        import traceback
        traceback.print_exc()
        return None

def run_batch_conversion(config):
    """Run batch conversion on all audio files in the input directory"""
    if not validate_config(config):
        return
    
    input_files = list_audio_files(config.batch_input_dir)
    if not input_files:
        return
    
    os.makedirs(config.batch_output_dir, exist_ok=True)
    
    success_count = 0
    fail_count = 0
    
    # Create progress bar
    pbar = tqdm(input_files, desc="Processing files")
    
    for input_file in pbar:
        filename = os.path.basename(input_file)
        name_without_ext = os.path.splitext(filename)[0]
        output_file = os.path.join(config.batch_output_dir, f"{name_without_ext}_converted.{config.export_format}")
        
        pbar.set_description(f"Processing {filename}")
        
        result = run_conversion(config, input_file, output_file)
        if result:
            success_count += 1
        else:
            fail_count += 1
    
    print(f"\nBatch conversion completed: {success_count} successful, {fail_count} failed")

## Workflow Guide

### Step 1: Download or Select a Model
Run the cell below to either download a model from a URL or select from available models.

In [ ]:
# Model Selection

# Option 1: Download a model from URL
model_url = ""  # Replace with your model URL
if model_url:
    download_model_from_url(model_url)
    # Refresh the model list
    available_models = list_available_models()

# Option 2: List available models
available_models = list_available_models()

# Option 3: Upload a model (Colab only)
# upload_model()

# Select a model (uncomment and modify as needed)
# if available_models:
#     selected_model = available_models[0]  # Select the first model
#     config.set_model(selected_model)

print("Model selection completed. Uncomment and modify the code above to select a model.")

### Step 2: Download or Select an Audio File
Run the cell below to either download audio from YouTube or select from available audio files.

In [ ]:
# Audio Selection

# Option 1: Download audio from YouTube
youtube_url = ""  # Replace with your YouTube URL
if youtube_url:
    downloaded_audio = download_youtube_audio(youtube_url)
    if downloaded_audio:
        config.set_input_audio(downloaded_audio)
    # Refresh the audio list
    input_audio_files = list_audio_files("audio_files/input")

# Option 2: List available audio files
input_audio_files = list_audio_files("audio_files/input")

# Option 3: Upload an audio file (Colab only)
# upload_audio()

# Select an audio file (uncomment and modify as needed)
# if input_audio_files:
#     selected_audio = input_audio_files[0]  # Select the first audio file
#     config.set_input_audio(selected_audio)

print("Audio selection completed. Uncomment and modify the code above to select an audio file.")

### Step 3: Configure Conversion Parameters
Run the cell below to set up your conversion parameters.

In [ ]:
# Parameter Setup

# Option 1: Load a preset
# config.update_from_preset("high_quality")
# config.update_from_preset("fast_conversion")

# Option 2: Set parameters manually
config.pitch = 0  # Pitch adjustment in semitones
config.index_rate = 0.75  # How much to use the index file (0.0-1.0)
config.f0_method = "rmvpe"  # Method for pitch extraction
config.protect = 0.33  # Protection for voiceless consonants
config.hop_length = 128  # Hop length for audio processing
config.split_audio = True  # Whether to split audio for processing large files
config.autotune = False  # Apply autotune
config.denoise = False  # Apply denoising
config.export_format = "wav"  # Output format

# Batch processing settings
config.batch_mode = False  # Set to True for batch processing
config.batch_input_dir = "audio_files/input"  # Directory with input files
config.batch_output_dir = "audio_files/output"  # Directory for output files

# Print current configuration
config.print_config()

# Validate configuration
is_valid = validate_config(config)

### Step 4: Preview Input Audio
Run the cell below to preview the input audio before conversion.

In [ ]:
# Preview Input Audio

if not config.batch_mode and config.input_audio_path and os.path.exists(config.input_audio_path):
    print("Input Audio Preview:")
    preview_audio(config.input_audio_path, "Input Audio")
elif config.batch_mode:
    print("Batch mode enabled - skipping individual audio preview")
else:
    print(f"Input audio file not found: {config.input_audio_path}")
    print("Please select an audio file in the previous step")

### Step 5: Run Conversion
Run the cell below to start the RVC conversion process.

In [ ]:
# Run Conversion

# Create a progress callback
def progress_callback(progress, message):
    print(f"[{progress*100:.0f}%] {message}")

if config.batch_mode:
    # Run batch conversion
    run_batch_conversion(config)
else:
    # Run single file conversion
    if is_valid:
        output_file = run_conversion(config, progress_callback=progress_callback)
        
        if output_file:
            print(f"\n✅ Conversion successful! Output saved to: {output_file}")
            
            # Preview the result
            print("\nOutput Audio Preview:")
            preview_audio(output_file, "Output Audio")
        else:
            print("\n❌ Conversion failed. Please check the configuration and try again.")
    else:
        print("\n❌ Configuration validation failed. Please fix the errors before running conversion.")

## Usage Instructions

### Basic Workflow
1. **Model Selection**: Download a model from URL or select from available models
2. **Audio Selection**: Download audio from YouTube or select from available files
3. **Parameter Setup**: Configure conversion parameters or load a preset
4. **Preview**: Preview the input audio before conversion
5. **Conversion**: Run the RVC conversion process

### Model Management
- Download models: `download_model_from_url(url)`
- List available models: `list_available_models()`
- Upload models (Colab): `upload_model()`
- Set model in config: `config.set_model(model_path)`

### Audio Management
- Download from YouTube: `download_youtube_audio(url)`
- List audio files: `list_audio_files(directory)`
- Upload audio (Colab): `upload_audio()`
- Set audio in config: `config.set_input_audio(audio_path)`
- Preview audio: `preview_audio(path, title)`

### Configuration
- Load presets: `config.update_from_preset(name)`
- Set parameters directly: `config.parameter = value`
- View configuration: `config.print_config()`
- Validate configuration: `validate_config(config)`

### Batch Processing
1. Set `config.batch_mode = True`
2. Set `config.batch_input_dir` to your input directory
3. Set `config.batch_output_dir` to your output directory
4. Run the conversion cell

### Parameters Explanation

#### Core Parameters
- `pitch`: Pitch adjustment in semitones (positive = higher, negative = lower)
- `index_rate`: How much to use the index file (0.0-1.0, higher = more faithful to model)
- `f0_method`: Method for pitch extraction ('harvest', 'crepe', 'rmvpe', etc.)
  - 'rmvpe': Best quality, slower
  - 'harvest': Good balance
  - 'crepe': Good for high-pitched voices
- `protect`: Protection for voiceless consonants (0.0-1.0, higher = more protection)

#### Advanced Parameters
- `hop_length`: Hop length for audio processing (lower = better quality, slower)
- `filter_radius`: Filter radius for audio processing
- `rms_mix_rate`: RMS mix rate (0.0-1.0)
- `split_audio`: Whether to split audio for processing large files
- `autotune`: Apply autotune to the output

#### Processing Options
- `denoise`: Apply denoising
- `deecho`: Apply deecho
- `reverb`: Apply reverb
- `use_tta`: Use Test Time Augmentation (better quality, slower)

### YouTube Download
- Cookie support: Place cookies in `./assets/ytdlstuff.txt`
- Supported formats: wav, mp3, flac, etc.
- Output directory: `audio_files/input`

### Troubleshooting

#### Common Issues
1. **Model not found**: Make sure the model_path is correct and the file exists
2. **Index file not found**: The index file should have the same name as the model but with .index extension
3. **Conversion fails**: Try reducing hop_length or changing f0_method
4. **Low quality output**: Try increasing index_rate or using a different f0_method
5. **Out of memory errors**: Try reducing batch_size or using CPU instead of GPU
6. **YouTube download fails**: Check your internet connection and cookies file

#### Performance Tips
- For faster conversion: Use 'harvest' f0_method, increase hop_length, disable split_audio
- For better quality: Use 'rmvpe' f0_method, decrease hop_length, enable use_tta
- For large files: Enable split_audio
- For YouTube downloads: Use cookies for age-restricted or region-locked content

## Note
This is a command-line style notebook with minimal UI elements. For more advanced features, use Advanced-RVC.ipynb